In [1]:
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import copy
import re
from time import sleep
from selenium import webdriver
import numpy as np

In [2]:
carmax_url = 'https://www.carmax.com/cars/ford'
#carmax_url = 'https://www.carmax.com/'
#carmax_url = 'https://www.carvana.com/cars/ford'
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [3]:
client = MongoClient('localhost', 27017)
db = client['capstone1']
pages = db['Ford_TX']

In [4]:
Ford_files=['Ford_2013_2016.html','Ford_2017_2020.html']

In [5]:
url_links = []
for file in Ford_files:
    soup = BeautifulSoup(open("/home/ali/Documents/CapStone_Projects/CapStone1/" + file), "html.parser")
    for idx,link in enumerate(soup.find_all('article','car-tile')):
        url_links.append(link.a['href'])
        

In [9]:
car_mileage = []
car_price = []
car_mpg = []
car_model_year = []
car_model = []
car_make = []
car_color = []
car_transmission = []
car_engine_size = []
car_horse_power = []
car_engine_cylinder = []
car_msrp_price = []

In [6]:
i=0

for link in set(url_links):
    scrape_url = link
    
    print(link,i)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
    
    r_1 = requests.get(scrape_url,headers=headers)
    
    soup = BeautifulSoup(r_1.text,'html.parser')
    
    car_mileage.append(soup.find_all('div','value')[1].text.strip())
    
    car_price.append(soup.find_all('div','value')[0].text.strip())
    
    car_mpg.append([np.nan if soup.find_all('div','mpg')==[] else soup.find('div','mpg').text])
    
    car_model_year.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[0])
    
    car_model.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[1])
    
    car_make.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__model-trim').text.strip())
    
    a = soup.find_all('script')[6]
    a = str(a)
    
    try:
        car_color.append(a.split('ExteriorColor":"')[1].split('"')[0])
    except Exception as e:
        car_color.append(np.nan)
    
    try:
        car_transmission.append(a.split('Transmission":"')[1].split('"')[0])
    except Exception as e:
        car_transmission.append(np.nan)
    
    try:
        car_engine_size.append(float(a.split('EngineSize":"')[1].split('L')[0]))
    except Exception as e:
        car_engine_size.append(np.nan)
    
    
    
    if a.split('Horsepower":')[1].split(',')[0] != 'null':
        var_HorsePower = float(a.split('Horsepower":')[1].split(',')[0])
    else:
        var_HorsePower = np.nan  
    car_horse_power.append(var_HorsePower)
    
    
    if a.split('EngineCylinders":')[1].split(',')[0] != 'null':
        car_EngineCylinder = int(a.split('EngineCylinders":')[1].split(',')[0])
    else:
        car_EngineCylinder = np.nan
    car_engine_cylinder.append(car_EngineCylinder)
    
    try:
        car_msrp_price.append(float(a.split('Msrp":')[1].split(',')[0]))
    except Exception as e:
        car_msrp_price.append(np.nan)
    
#    pages.insert_one({'car_mileage': car_mileage, 
#                      'car_price': car_price, 
#                      'car_mpg': car_mpg ,
#                      'car_model_year': car_model_year,
#                      'car_model': car_model ,
#                      'car_make': car_make,
#                      'car_color': car_color,
#                      'car_transmission': car_transmission,
#                      'car_engine_size': car_engine_size,
#                      'car_horse_power': car_horse_power ,
#                      'car_engine_cylinder': car_engine_cylinder,
#                      'car_msrp_price':car_msrp_price})
    
    
    i += 1
    sleep(10)
    if i>200:
        break 

https://www.carmax.com/car/19518507
https://www.carmax.com/car/19686642
https://www.carmax.com/car/19888650
https://www.carmax.com/car/19510815
https://www.carmax.com/car/18832345
https://www.carmax.com/car/19269216
https://www.carmax.com/car/18266755
https://www.carmax.com/car/19539061
https://www.carmax.com/car/19651577
https://www.carmax.com/car/19860269
https://www.carmax.com/car/20096806
https://www.carmax.com/car/19981448
https://www.carmax.com/car/19649379
https://www.carmax.com/car/19723223
https://www.carmax.com/car/19859471
https://www.carmax.com/car/20050881
https://www.carmax.com/car/19709370
https://www.carmax.com/car/20061790
https://www.carmax.com/car/19858782
https://www.carmax.com/car/19651501
https://www.carmax.com/car/19694366
https://www.carmax.com/car/19990875
https://www.carmax.com/car/19970539
https://www.carmax.com/car/19538523
https://www.carmax.com/car/18536686
https://www.carmax.com/car/19860076
https://www.carmax.com/car/19875743
https://www.carmax.com/car/1

In [7]:
df = pd.DataFrame({'car_mileage': car_mileage, 
                   'car_price': car_price, 
                   'car_mpg': car_mpg ,
                   'car_model_year': car_model_year,
                   'car_model': car_model ,
                   'car_make': car_make,
                   'car_color': car_color,
                   'car_transmission': car_transmission,
                   'car_engine_size': car_engine_size,
                   'car_horse_power': car_horse_power ,
                   'car_engine_cylinder': car_engine_cylinder,
                   'car_msrp_price':car_msrp_price,
                                          })
df

,car_mileage,car_price,car_mpg,car_model_year,car_model,car_make,car_color,car_transmission,car_engine_size,car_horse_power,car_engine_cylinder,car_msrp_price
0,22K miles,"$32,998*",[nan],2019,F150 XL,Ford,Gray,Automatic,2.7,325.0,6,38300.0
1,110K miles,"$10,998*",[23/36 mpg],2014,Fusion SE,Ford,Red,Automatic,1.5,178.0,4,24000.0
2,40K miles,"$23,998*",[nan],2019,Ranger XL,Ford,Gray,Automatic,2.3,NaN,4,26700.0
3,34K miles,N/A*,[43/41 mpg],2019,Fusion Hybrid SE,Ford,Silver,Automatic,2.0,188.0,4,27600.0
4,96K miles,N/A*,[nan],2011,Mustang,Ford,White,Automatic,3.7,305.0,6,22200.0
...,...,...,...,...,...,...,...,...,...,...,...,...
196,42K miles,N/A*,[nan],2019,Fusion Hybrid SE,Ford,Silver,Automatic,2.0,NaN,4,27600.0
197,4K miles,N/A*,[27/29 mpg],2019,EcoSport SE,Ford,Black,Automatic,1.0,123.0,3,23200.0
198,13K miles,"$25,998*",[22/29 mpg],2019,Edge SEL,Ford,Red,Automatic,2.0,245.0,4,33300.0
199,48K miles,"$19,998*",[nan],2017,Explorer,Ford,Black,Automatic,2.3,NaN,4,31700.0


In [8]:
#df.to_csv("Ford_TX.csv")

In [ ]:
#xxx= pd.read_csv('Ford_TX.csv')

In [ ]:
#xxx